In [12]:
from data import imagenet as imgnt
from utils.evaluate import evaluate, Accuracy

import torch
import timm

import random
import inspect


from PIL import Image
import requests
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import torchvision
import torchvision.transforms as T

from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

torch.set_grad_enabled(False);

from timm.models.vision_transformer import VisionTransformer


In [2]:
dataset_path = "/scratch_shared/primmere/ILSVRC/Data/CLS-LOC"
imagenet = imgnt.ImageNet(dataset_path, 1)
val = imagenet.get_valid_set()
device = torch.device("cuda")

model = timm.create_model('deit_tiny_patch16_224.fb_in1k', pretrained=True)
model.to(device)
model.eval()
next(model.parameters()).is_cuda 
val

Dataset ImageNetDataset
    Number of datapoints: 50000
    Root location: /scratch_shared/primmere/ILSVRC/Data/CLS-LOC/val
    Compose(
    ToTensor()
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [3]:
indices = random.sample(range(50_000),1000)
#indices = imgnt.get_sample_indices_for_class(val, list(range(100)), 10_000, device)
val_small = imgnt.ImageNetSubset(val,indices)


In [4]:
val_loader = torch.utils.data.DataLoader(val_small,8, pin_memory=True)
img, label = next(iter(val_loader))


In [5]:
img = img.cuda()
img.device
img.shape

torch.Size([8, 3, 224, 224])

In [6]:
label

tensor([ 55,  61, 428, 296, 769, 349, 728, 766])

In [7]:
torch.argmax(model(img), dim=1)

tensor([ 64,  61, 756, 296, 769, 348, 728, 766], device='cuda:0')

In [8]:
acc = Accuracy()
results = evaluate(model, val_loader, acc, device)

Evaluating: 100%|██████████| 125/125 [00:08<00:00, 14.52batch/s]


In [9]:
print(results['confusion_matrix'])
print(results['total_accuracy'])

[[2 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 2 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]
0.724


In [13]:
print(inspect.getsource(VisionTransformer.__init__))  # constructor w/ blocks & heads
print(inspect.getsource(VisionTransformer.forward))   # forward pass
print(inspect.getsourcefile(VisionTransformer))       # file path

    def __init__(
            self,
            img_size: Union[int, Tuple[int, int]] = 224,
            patch_size: Union[int, Tuple[int, int]] = 16,
            in_chans: int = 3,
            num_classes: int = 1000,
            global_pool: Literal['', 'avg', 'avgmax', 'max', 'token', 'map'] = 'token',
            embed_dim: int = 768,
            depth: int = 12,
            num_heads: int = 12,
            mlp_ratio: float = 4.,
            qkv_bias: bool = True,
            qk_norm: bool = False,
            proj_bias: bool = True,
            init_values: Optional[float] = None,
            class_token: bool = True,
            pos_embed: str = 'learn',
            no_embed_class: bool = False,
            reg_tokens: int = 0,
            pre_norm: bool = False,
            final_norm: bool = True,
            fc_norm: Optional[bool] = None,
            dynamic_img_size: bool = False,
            dynamic_img_pad: bool = False,
            drop_rate: float = 0.,
            pos_d